In [46]:
import nltk
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import unicodedata
import string
import os
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
df1 = pd.read_json('sandiego2.json', orient = 'table')

In [30]:
df2 = pd.read_json('inlandempire2.json', orient = 'table')

In [32]:
print(len(df2.dropna()))

1092


In [33]:
len(df2)

1092

In [34]:
dfx = df2[df2['price'] != '']

In [35]:
dfx

,posted,neighborhood,title,url,price,roomstext
2,2020-12-01 13:25,(Las Vegas Strip Area),Get out of Quarantine - Move to Vegas,https://inlandempire.craigslist.org/roo/d/las-...,1,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nGet out of...
3,2020-12-01 13:05,"(Palomar St Wildomar, CA 92595)",Beautiful Room for Rent in the Country,https://inlandempire.craigslist.org/roo/d/wild...,400,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBeautiful ...
4,2020-12-01 13:03,"(Almería, between foothill and baseline)",Recámara en renta!!!,https://inlandempire.craigslist.org/roo/d/font...,1200,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRecamara e...
6,2020-12-01 12:57,(Pomona),Room for Rent,https://inlandempire.craigslist.org/roo/d/pomo...,650,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRoom for R...
8,2020-12-01 12:26,(Eastvale),Furnished private room in a beautiful house,https://inlandempire.craigslist.org/roo/d/chin...,950,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFurnished ...
...,...,...,...,...,...,...
1084,2020-10-18 01:12,(Border of Cacumonga/North Fontana),Male/Furnished/Utilities Covered/WiFi,https://inlandempire.craigslist.org/roo/d/font...,600,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMale/Furni...
1086,2020-10-17 20:56,(SAN BERNADINO),FURN ROOM/ 4 FULL TIME WORK PROFESSIONAL,https://inlandempire.craigslist.org/roo/d/san-...,600,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFURN ROOM/...
1088,2020-10-17 15:02,(Moreno valley),Room for rent,https://inlandempire.craigslist.org/roo/d/more...,450,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRoom for r...
1089,2020-10-17 10:45,(Riverside),room for rent,https://inlandempire.craigslist.org/roo/d/rive...,700,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nroom for r...


In [7]:
areas = ['sfbay', 'sanantonio', 'albuquerque', 'yuma', 'lasvegas', 'elpaso', 'bellingham', 'boise', 'lascruces', 'phoenix', 'buffalo', 'detroit', 'chicago', 'kansascity', 'stlouis', 'denver', 'reno', 'sacramento', 'stockton', 'seattle', 'losangeles',]
areas2 = ['washingtondc', 'baltimore', 'charlotte', 'atlanta', 'plattsburgh', 'miami', 'jacksonville', 'vermont', 'nh', 'philadelphia', 'pittsburgh', 'delaware', 'maine', 'bozeman', 'missoula', 'neworleans', 'shreveport', 'louisville', 'memphis', 'omaha', 'wichita', 'eugene', 'humboldt', 'slo', 'santabarbara', 'bakersfield', 'fresno', 'newhaven']


In [9]:
df = pd.concat([df1, df2], ignore_index = True)

In [10]:
df

,posted,neighborhood,title,url,price,roomstext
0,2020-12-01 14:29,(Mission Hills),Room for Rent,https://sandiego.craigslist.org/csd/roo/d/san-...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRoom for Ren...
1,2020-12-01 14:19,(Encintas),"RARE FIND! Newly renovated NICE spot, walk to ...",https://sandiego.craigslist.org/nsd/roo/d/carl...,1600,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRARE FIND!...
2,2020-12-01 14:19,"(SDSU, College Area)",Short Walk To SDSU & Trolley. Female Share Lg ...,https://sandiego.craigslist.org/csd/roo/d/san-...,600,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nShort Walk...
3,2020-12-01 14:14,(Point Loma),1Bed 1 Bath in large home with view,https://sandiego.craigslist.org/csd/roo/d/san-...,995,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n1Bed 1 Bat...
4,2020-12-01 14:08,"(College East, SDSU, San Diego)",Private bedroom available,https://sandiego.craigslist.org/csd/roo/d/san-...,1000,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPrivate be...
...,...,...,...,...,...,...
2080,2020-10-17 17:22,(Victorville),Room and board for help around house,https://inlandempire.craigslist.org/roo/d/vict...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRoom and boa...
2081,2020-10-17 15:02,(Moreno valley),Room for rent,https://inlandempire.craigslist.org/roo/d/more...,450,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRoom for r...
2082,2020-10-17 10:45,(Riverside),room for rent,https://inlandempire.craigslist.org/roo/d/rive...,700,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nroom for r...
2083,2020-10-17 10:24,(Ontario),Male looking for female who needs a place to stay,https://inlandempire.craigslist.org/roo/d/onta...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMale looking...


In [11]:
df1 = df

In [13]:
df2 = pd.read_json('newyork2.json', orient = 'table')

In [14]:
df = pd.concat([df1, df2], ignore_index = True)

In [17]:
df1 = df

In [19]:
df1.iloc[2000:2100]

,posted,neighborhood,title,url,price,roomstext
2000,2020-10-22 16:58,(Eagle Glen),Room for rent in very clean suburbs,https://inlandempire.craigslist.org/roo/d/coro...,600,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRoom for r...
2001,2020-10-22 16:49,(Victorville),3 Private Bedrooms in 5BR/4BA House,https://inlandempire.craigslist.org/roo/d/vict...,550,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n3 Private ...
2002,2020-10-22 16:49,(Hemet),One bedroom for rent,https://inlandempire.craigslist.org/roo/d/heme...,650,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nOne bedroo...
2003,2020-10-22 16:45,(HESPERIA),Space for Rv,https://inlandempire.craigslist.org/roo/d/hesp...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSpace for Rv...
2004,2020-10-22 15:01,(Rancho Cucamonga),Room For Rent Own Entrance Own Bathroom $850.00,https://inlandempire.craigslist.org/roo/d/ranc...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n try the crai...
...,...,...,...,...,...,...
2095,2020-12-01 17:35,(Inwood / Wash Hts),"Master bedroom, with private bathroom and fire...",https://newyork.craigslist.org/mnh/roo/d/new-y...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMaster bedro...
2096,2020-12-01 17:31,(Upper West Side),Newly Renovated Apt.. 1BR Available.,https://newyork.craigslist.org/mnh/roo/d/new-y...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nNewly Renova...
2097,2020-12-01 17:30,(Chelsea),3br - 1300ft2 - FURNISHED 1 LRG BDRM in Lux Ch...,https://newyork.craigslist.org/mnh/roo/d/new-y...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n3br - 1300ft...
2098,2020-12-01 17:29,(BRONX NY BRONX NY BRONX NY BRONX NY BRONX NY...,ROOM FOR RENT IN NICE QUITE NEIGHBORHOOD SINGL...,https://newyork.craigslist.org/brx/roo/d/bronx...,0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nROOM FOR R...


In [20]:
df2 = pd.read_json('dallas2.json', orient = 'table')

In [21]:
df = pd.concat([df1, df2], ignore_index = True)

In [22]:
df

,posted,neighborhood,title,url,price,roomstext
0,2020-12-01 14:29,(Mission Hills),Room for Rent,https://sandiego.craigslist.org/csd/roo/d/san-...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRoom for Ren...
1,2020-12-01 14:19,(Encintas),"RARE FIND! Newly renovated NICE spot, walk to ...",https://sandiego.craigslist.org/nsd/roo/d/carl...,1600,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRARE FIND!...
2,2020-12-01 14:19,"(SDSU, College Area)",Short Walk To SDSU & Trolley. Female Share Lg ...,https://sandiego.craigslist.org/csd/roo/d/san-...,600,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nShort Walk...
3,2020-12-01 14:14,(Point Loma),1Bed 1 Bath in large home with view,https://sandiego.craigslist.org/csd/roo/d/san-...,995,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n1Bed 1 Bat...
4,2020-12-01 14:08,"(College East, SDSU, San Diego)",Private bedroom available,https://sandiego.craigslist.org/csd/roo/d/san-...,1000,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPrivate be...
...,...,...,...,...,...,...
6083,2020-10-17 15:10,(Dallas),Male nudist roommate needed,https://dallas.craigslist.org/dal/roo/d/dallas...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMale nudist ...
6084,2020-10-17 14:39,(Commerce),Master bedroom w/ walk-in closet and bathroom,https://dallas.craigslist.org/mdf/roo/d/commer...,400,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMaster bed...
6085,2020-10-17 14:32,(UT CAMPUS),**College room near UT now**,https://dallas.craigslist.org/sdf/roo/d/dallas...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n**College ro...
6086,2020-10-17 13:16,(Arlington),Master bedroom w bath convenience to 360 and I20!,https://dallas.craigslist.org/dal/roo/d/arling...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMaster bedro...


In [23]:
df1 = df

In [24]:
for area in areas:
    file = area + '2.json'
    df2 = pd.read_json(file, orient = 'table')
    df = pd.concat([df1, df2], ignore_index = True)
    df1 = df

In [26]:
df1

,posted,neighborhood,title,url,price,roomstext
0,2020-12-01 14:29,(Mission Hills),Room for Rent,https://sandiego.craigslist.org/csd/roo/d/san-...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRoom for Ren...
1,2020-12-01 14:19,(Encintas),"RARE FIND! Newly renovated NICE spot, walk to ...",https://sandiego.craigslist.org/nsd/roo/d/carl...,1600,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRARE FIND!...
2,2020-12-01 14:19,"(SDSU, College Area)",Short Walk To SDSU & Trolley. Female Share Lg ...,https://sandiego.craigslist.org/csd/roo/d/san-...,600,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nShort Walk...
3,2020-12-01 14:14,(Point Loma),1Bed 1 Bath in large home with view,https://sandiego.craigslist.org/csd/roo/d/san-...,995,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n1Bed 1 Bat...
4,2020-12-01 14:08,"(College East, SDSU, San Diego)",Private bedroom available,https://sandiego.craigslist.org/csd/roo/d/san-...,1000,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPrivate be...
...,...,...,...,...,...,...
26786,2020-11-24 09:19,(North Kings Road),Great 2BR 2BA to share in Prime WeHo (This is...,https://losangeles.craigslist.org/wst/roo/d/gr...,1550,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nGreat 2BR ...
26787,2020-11-24 09:13,(Santa Monica),"Private Room, 7 blocks from the beach, AMAZING...",https://losangeles.craigslist.org/wst/roo/d/sa...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPrivate Room...
26788,2020-11-24 09:11,(LA),Studio/Share Couch 🏄‍♀️,https://losangeles.craigslist.org/lac/roo/d/lo...,300,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nStudio/Sha...
26789,2020-11-24 09:09,(Windsor Hills),Master Bedroom in Shared House,https://losangeles.craigslist.org/wst/roo/d/lo...,895,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMaster Bed...


In [36]:
df1

,posted,neighborhood,title,url,price,roomstext
0,2020-12-01 14:29,(Mission Hills),Room for Rent,https://sandiego.craigslist.org/csd/roo/d/san-...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRoom for Ren...
1,2020-12-01 14:19,(Encintas),"RARE FIND! Newly renovated NICE spot, walk to ...",https://sandiego.craigslist.org/nsd/roo/d/carl...,1600,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRARE FIND!...
2,2020-12-01 14:19,"(SDSU, College Area)",Short Walk To SDSU & Trolley. Female Share Lg ...,https://sandiego.craigslist.org/csd/roo/d/san-...,600,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nShort Walk...
3,2020-12-01 14:14,(Point Loma),1Bed 1 Bath in large home with view,https://sandiego.craigslist.org/csd/roo/d/san-...,995,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n1Bed 1 Bat...
4,2020-12-01 14:08,"(College East, SDSU, San Diego)",Private bedroom available,https://sandiego.craigslist.org/csd/roo/d/san-...,1000,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPrivate be...
...,...,...,...,...,...,...
26786,2020-11-24 09:19,(North Kings Road),Great 2BR 2BA to share in Prime WeHo (This is...,https://losangeles.craigslist.org/wst/roo/d/gr...,1550,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nGreat 2BR ...
26787,2020-11-24 09:13,(Santa Monica),"Private Room, 7 blocks from the beach, AMAZING...",https://losangeles.craigslist.org/wst/roo/d/sa...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPrivate Room...
26788,2020-11-24 09:11,(LA),Studio/Share Couch 🏄‍♀️,https://losangeles.craigslist.org/lac/roo/d/lo...,300,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nStudio/Sha...
26789,2020-11-24 09:09,(Windsor Hills),Master Bedroom in Shared House,https://losangeles.craigslist.org/wst/roo/d/lo...,895,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMaster Bed...


In [37]:
for area in areas2:
    file = area + '2.json'
    df2 = pd.read_json(file, orient = 'table')
    df = pd.concat([df1, df2], ignore_index = True)
    df1 = df

In [39]:
df

,posted,neighborhood,title,url,price,roomstext
0,2020-12-01 14:29,(Mission Hills),Room for Rent,https://sandiego.craigslist.org/csd/roo/d/san-...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRoom for Ren...
1,2020-12-01 14:19,(Encintas),"RARE FIND! Newly renovated NICE spot, walk to ...",https://sandiego.craigslist.org/nsd/roo/d/carl...,1600,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRARE FIND!...
2,2020-12-01 14:19,"(SDSU, College Area)",Short Walk To SDSU & Trolley. Female Share Lg ...,https://sandiego.craigslist.org/csd/roo/d/san-...,600,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nShort Walk...
3,2020-12-01 14:14,(Point Loma),1Bed 1 Bath in large home with view,https://sandiego.craigslist.org/csd/roo/d/san-...,995,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n1Bed 1 Bat...
4,2020-12-01 14:08,"(College East, SDSU, San Diego)",Private bedroom available,https://sandiego.craigslist.org/csd/roo/d/san-...,1000,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPrivate be...
...,...,...,...,...,...,...
39924,2020-10-19 14:54,"(Shelton, CT)",ROOM FOR RENT SHELTON $175/WEEK EVERYTHING INC...,https://newhaven.craigslist.org/roo/d/derby-ro...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nROOM FOR REN...
39925,2020-10-19 12:16,(Cheshire),Responsible Young Tenant for Room to Rent,https://newhaven.craigslist.org/roo/d/cheshire...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nResponsible ...
39926,2020-10-18 21:23,(Wallingford),Wallingford room for rent,https://newhaven.craigslist.org/roo/d/wallingf...,650,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nWallingfor...
39927,2020-10-18 18:46,"(Park Ave, Bridgeport)","3 Story Townhouse (3BR, 2.5B) 2 Bedrooms Avai...",https://newhaven.craigslist.org/roo/d/bridgepo...,650,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n3 Story To...


In [40]:
dfbackup = df

In [41]:
dfbackup

,posted,neighborhood,title,url,price,roomstext
0,2020-12-01 14:29,(Mission Hills),Room for Rent,https://sandiego.craigslist.org/csd/roo/d/san-...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRoom for Ren...
1,2020-12-01 14:19,(Encintas),"RARE FIND! Newly renovated NICE spot, walk to ...",https://sandiego.craigslist.org/nsd/roo/d/carl...,1600,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRARE FIND!...
2,2020-12-01 14:19,"(SDSU, College Area)",Short Walk To SDSU & Trolley. Female Share Lg ...,https://sandiego.craigslist.org/csd/roo/d/san-...,600,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nShort Walk...
3,2020-12-01 14:14,(Point Loma),1Bed 1 Bath in large home with view,https://sandiego.craigslist.org/csd/roo/d/san-...,995,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n1Bed 1 Bat...
4,2020-12-01 14:08,"(College East, SDSU, San Diego)",Private bedroom available,https://sandiego.craigslist.org/csd/roo/d/san-...,1000,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPrivate be...
...,...,...,...,...,...,...
39924,2020-10-19 14:54,"(Shelton, CT)",ROOM FOR RENT SHELTON $175/WEEK EVERYTHING INC...,https://newhaven.craigslist.org/roo/d/derby-ro...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nROOM FOR REN...
39925,2020-10-19 12:16,(Cheshire),Responsible Young Tenant for Room to Rent,https://newhaven.craigslist.org/roo/d/cheshire...,,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nResponsible ...
39926,2020-10-18 21:23,(Wallingford),Wallingford room for rent,https://newhaven.craigslist.org/roo/d/wallingf...,650,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nWallingfor...
39927,2020-10-18 18:46,"(Park Ave, Bridgeport)","3 Story Townhouse (3BR, 2.5B) 2 Bedrooms Avai...",https://newhaven.craigslist.org/roo/d/bridgepo...,650,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n3 Story To...


In [42]:
df = df[df['price'] != '']

In [43]:
df

,posted,neighborhood,title,url,price,roomstext
1,2020-12-01 14:19,(Encintas),"RARE FIND! Newly renovated NICE spot, walk to ...",https://sandiego.craigslist.org/nsd/roo/d/carl...,1600,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRARE FIND!...
2,2020-12-01 14:19,"(SDSU, College Area)",Short Walk To SDSU & Trolley. Female Share Lg ...,https://sandiego.craigslist.org/csd/roo/d/san-...,600,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nShort Walk...
3,2020-12-01 14:14,(Point Loma),1Bed 1 Bath in large home with view,https://sandiego.craigslist.org/csd/roo/d/san-...,995,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n1Bed 1 Bat...
4,2020-12-01 14:08,"(College East, SDSU, San Diego)",Private bedroom available,https://sandiego.craigslist.org/csd/roo/d/san-...,1000,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPrivate be...
5,2020-12-01 14:08,"(College East, San Diego)",Private bedroom,https://sandiego.craigslist.org/csd/roo/d/san-...,800,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPrivate be...
...,...,...,...,...,...,...
39917,2020-10-21 10:57,(Waterbury),Room for Rent,https://newhaven.craigslist.org/roo/d/waterbur...,700,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRoom for R...
39918,2020-10-21 09:34,(WATERBURY),Roommate Wanted,https://newhaven.craigslist.org/roo/d/waterbur...,800,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRoommate W...
39921,2020-10-20 12:28,(Bridgeport),One Bedroom,https://newhaven.craigslist.org/roo/d/bridgepo...,800,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nOne Bedroo...
39926,2020-10-18 21:23,(Wallingford),Wallingford room for rent,https://newhaven.craigslist.org/roo/d/wallingf...,650,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nWallingfor...


In [44]:
X = df['roomstext']
y = df['price']

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [49]:
len(X_train)

19965

In [50]:
len(y_train)

19965

In [51]:
def removeaccents(text):
    nfkdform = unicodedata.normalize('NFKD', text)
    asctext = nfkdform.encode('ASCII', 'ignore')
    return asctext.decode()

In [52]:
def filter_tokens(tokens):
    return([w for w in tokens if not w in stopwords_ and not w in punctuation_])

In [54]:
X_train[1]

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRARE FIND! Newly renovated NICE spot, walk to the beach! - rooms &...\n\n \n\n\n\n\n\n\n\n\n    try the craigslist app \n    \n        Android\n    \n\n        iOS\n    \n\n\nCL\n\n\n\n\nsan diego\n>\n\n\n\n\nnorth SD county\n>\n\n\n\n\nhousing\n>\n\n\n\n\nrooms & shares\n\n\n\n\n\n\n\npost\n\n\naccount\n\n\n\n\n\n favorites\n\n\n\n\n\n\n\n\n\n\n hidden\n\n\n\n\n\n\n\n\nCL\n\n\n\nnorth SD county            >\n\nrooms & shares\n\n...\n\n\n\n\n\n  prev \n\n next  \n\n\n\n\n\n      reply\n    \n\n\n\n\nfavorite\n\n\n\nfavorite\n\n\n\n\n\nhide\n\n\n\nunhide\n\n\n\n\n\n\n\n\n\n                flag\n            \n\n\n\n\n                flagged\n            \n\n\n\n\n                    Posted\n                    \n                        2020-12-01 14:15\n                    \n\n\n                Contact Information:  \n\nprint\n\n\n\n\n$1,600 / 1br - RARE FIND! Newly renovated NICE spot, walk to the beach! (Encintas) \n\n\n\n\n<\nimage 1 of 12\n>\n\n\n

In [55]:
    termslist = []
    corpusvocab = set()
    paragraphs = X_train
    documents = []
    for paragraph in paragraphs:
        documents.append(str(paragraph))
    texts = []
    for document in documents:
        soup = BeautifulSoup(document, 'html.parser')
        texts.append(soup.get_text())
    
    ratexts = []
    for text in texts:
        txt = removeaccents(text)
        ratexts.append(txt)

    
    for text in ratexts:
        senttokens = nltk.tokenize.sent_tokenize(text)
        tokens = [sent for sent in map(nltk.tokenize.word_tokenize, senttokens)]
        lower = [[word.lower() for word in sent] for sent in tokens]
        stopwords_ = set(nltk.corpus.stopwords.words('english'))
        stopwords_.discard('not')
        punctuation_ = set(string.punctuation)
        filteredtokens = list(map(filter_tokens, lower))
        stemporter = nltk.stem.porter.PorterStemmer()
        stemtokens = [list(map(stemporter.stem, word))for word in filteredtokens]
        terms = []
        vocab = set()
        for stems in stemtokens:
            for word in stems:
                vocab.add(word)
                terms.append(word)
        termslist.append(terms)
        corpusvocab.update(vocab)

In [56]:
len(termslist)

19965

In [57]:
len(corpusvocab)

72871

In [58]:
type(corpusvocab)

set

In [59]:
termoccur = list(map(lambda term : Counter(term), termslist))

In [60]:
len(termoccur)

19965

In [61]:
termoccur[1]

Counter({'rent': 3,
         'master': 4,
         'bedroom': 5,
         'room': 7,
         'share': 3,
         'apart': 1,
         'roommat': 3,
         '...': 2,
         'tri': 1,
         'craigslist': 1,
         'app': 1,
         'android': 1,
         'io': 1,
         'cl': 2,
         'fresno': 2,
         'hous': 6,
         'post': 6,
         'account': 1,
         'favorit': 3,
         'hidden': 1,
         'prev': 1,
         'next': 1,
         'repli': 1,
         'hide': 1,
         'unhid': 1,
         'flag': 3,
         '2020-11-28': 2,
         '09:33': 2,
         'contact': 2,
         'inform': 1,
         'print': 1,
         '750': 2,
         'clovi': 2,
         'imag': 1,
         '1': 1,
         '11': 1,
         '2197': 1,
         'mitchel': 1,
         'ave': 1,
         'near': 1,
         'armstrong': 2,
         'barstow': 2,
         'googl': 1,
         'map': 1,
         'cat': 1,
         'ok': 2,
         'purrr': 1,
         'dog': 2,
 

In [63]:
termfreq = []
for i in range(len(termoccur)):
    termfreq.append({k : (v / float(len(termslist[i]))) 
                    for k, v in termoccur[i].items()})

In [64]:
len(termfreq)

19965

In [65]:
dococcur = Counter( [word for term in termslist for word in set(term)])

In [68]:
type(dococcur)

collections.Counter

In [76]:
docfreq = {k:(v/float(len(termslist))) for k,v in dococcur.items() }

In [85]:
mindf = .002
vocab = [k for k, v in docfreq.items() if v > mindf]

In [86]:
len(vocab)

3586

In [88]:
vectors = np.zeros((len(termslist), len(vocab)))

In [87]:
19965 * 4000

79860000

In [89]:
for i in range(len(termslist)):
    for j in range(len(vocab)):
        tempterm = vocab[j]
        temptermf = termfreq[i].get(tempterm, 0.0)
        termidf = np.log(1+1/docfreq[tempterm])
        vectors[i,j] = temptermf * termidf

In [90]:
np.save('tfidf.npy', vectors)

In [91]:
type(X_train)

pandas.core.series.Series

In [92]:
X_train.to_json('X_train.json', orient = 'table')

In [93]:
X_test.to_json('X_test.json', orient = 'table')

In [94]:
y_train.to_json('y_train.json', orient = 'table')

In [95]:
y_test.to_json('y_test.json', orient = 'table')

In [96]:
X_traindf = pd.DataFrame(columns = ['stemtext', 'tfidf'])

In [97]:
X_traindf['stemtext'] = X_train 

In [ ]:
X_traindf['tfidf'] = vectors